In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
articles_raw = pd.read_csv("use_rEx.csv") #Do not run more than once - hogs memory and takes some time
articles_raw.head()

,author,contents,description,publisher,source_url,timeStamp,title
14599,Stripe.com,A complete payments platform engineered for gr...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support
14591,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: Trudeau announces pacific trade de...
14592,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: Trump to tell Davos the US 'open f...
14593,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: VR film in Davos highlights cyber ...
14594,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: IMF's Lagarde faults Modi speech i...


In [3]:
articles_raw.tail()

,author,contents,description,publisher,source_url,timeStamp,title
28959,Avivah Litan,\tA member of the Gartner Blog Network\tby Avi...,Blockchain sessions were in full force at the ...,Gartner.com,https://blogs.gartner.com/avivah-litan/2018/04...,2018-04-23 23:22:25+00:00,"Blockchain, Crypto, and Women; Musings from RSAC"
28960,Cyberparse.co.uk,Enlarge / Simplified figurative process of a C...,Enlarge / Simplified figurative process of a C...,Cyberparse.co.uk,http://cyberparse.co.uk/2018/04/23/new-hacks-s...,2018-04-23 23:32:00+00:00,New hacks siphon private cryptocurrency keys f...
28961,Ali Breland and Harper Neidig,THE EU'S NEW TARGET: They're back! European re...,THE EU'S NEW TARGET: They're back! European re...,The Hill,http://thehill.com/policy/technology/overnight...,2018-04-23 23:49:29+00:00,Overnight Tech: EU investigates Apple's Shazam...
28962,Tyler Durden,Surprise! Even kids can be taught about finan...,"Authored by Caitlin Johnstone via Medium.com, ...",Zerohedge.com,https://www.zerohedge.com/news/2018-04-23/msm-...,2018-04-23 23:55:00+00:00,MSM Is Frantically Attacking Dissenting Syria ...
28963,pjbyrne,"Disclaimer: English lawyer, not practising thi...","Disclaimer: English lawyer, not practising thi...",Prestonbyrne.com,https://prestonbyrne.com/2018/04/23/on-ethereu...,2018-04-23 23:59:35+00:00,Whether Ethereum is a security


In [4]:
articles_raw.describe()

,author,contents,description,publisher,source_url,timeStamp,title
count,28601,28774,28770,28774,28774,28774,28764
unique,7079,26417,27539,1740,27733,26813,25302
top,Scott Scanlon,If you are the site owner (or you manage this ...,"Welcome to Crypto Insider, Business Insider's ...",Youbrandinc.com,https://www.bloomberg.com/news/articles/2018-0...,2018-01-23 00:00:00+00:00,6 things Australian traders will be talking ab...
freq,1743,57,5,1730,3,10,43


In [5]:
#Reset index
articles_raw = articles_raw.reset_index()
articles_raw = articles_raw.drop('index', axis = 1)
articles_raw.head()

,author,contents,description,publisher,source_url,timeStamp,title
0,Stripe.com,A complete payments platform engineered for gr...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support
1,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: Trudeau announces pacific trade de...
2,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: Trump to tell Davos the US 'open f...
3,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: VR film in Davos highlights cyber ...
4,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: IMF's Lagarde faults Modi speech i...


#### Methodology: 
We have a list of keywords (defined below) that we deem relevant to Bitcoin trading and price. If an article does not mention one of these keywords in the first 300 characters (heuristic/empirical analysis of where the lede of an article ends) we deem it irrelevant.

In [10]:
#Sample of first 300 characters of the first 30 articles. 
#As you can see, relevant articles will mention Bitcoin or a keyword in this selection.
for article in articles_raw['contents'][:30]:
    print(article[:300])
    print()

A complete payments platform engineered for growth.	Build and scale your recurring business model.	Everything platforms need to get sellers paid.	Your business data at your fingertips.	The best way to start an internet business.	Fight fraud with machine learning.	Share this post on Twitter	
Tom Karl

DAVOS, Switzerland  — The latest on the World Economic Forum in Davos, Switzerland (all times local):	11:20 p.m.
  	President Donald Trump plans to meet with world leaders, dine with business executives and declare America "open for business" during his trip to the Davos summit.	White House official

DAVOS, Switzerland  — The latest on the World Economic Forum in Davos, Switzerland (all times local):	11:20 p.m.
  	President Donald Trump plans to meet with world leaders, dine with business executives and declare America "open for business" during his trip to the Davos summit.	White House official

DAVOS, Switzerland  — The latest on the World Economic Forum in Davos, Switzerland (all times 

In [30]:
#Define list of keywords and new 
keywords = [ 
            "BTC", "btc", "BCH", "bch", "Bitcoin", "bitcoin", \
            "Litecoin", "litecoin", "LTC", "ltc", \
            "Ether", "ether", "Ethereum", "ethereum", \
            "ETH", "eth", \
            "hodl", "HODL", \
            "crypto", "cryptocurrency", "cryptocurrencies", \
            "Crypto", "Cryptocurrency", "Cryptocurrencies", \
            "ICO", "ico", "GDAX", "gdax", \
            "Blockchain", "blockchain", \
           ]

articles_final = articles_raw
articles_final.head()

,author,contents,description,publisher,source_url,timeStamp,title
0,Stripe.com,A complete payments platform engineered for gr...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support
1,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: Trudeau announces pacific trade de...
2,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: Trump to tell Davos the US 'open f...
3,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: VR film in Davos highlights cyber ...
4,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: IMF's Lagarde faults Modi speech i...


In [31]:
articles_final.head()

,author,contents,description,publisher,source_url,timeStamp,title
0,Stripe.com,A complete payments platform engineered for gr...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support
1,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: Trudeau announces pacific trade de...
2,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: Trump to tell Davos the US 'open f...
3,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: VR film in Davos highlights cyber ...
4,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: IMF's Lagarde faults Modi speech i...


In [53]:
#Traverse the articles array. 
#If you find an article that does not contain a keyword in the first 300 characters, remove it.
n_removed = 0
indices_to_drop = []

for i in articles_final.index:
    first300 = articles_final['contents'][i][:300] #Grab first 300 characters of article text
    title = articles_final['title'][i]
    n_keywords = 0
    try:
        assert type(first300) is str
        assert type(title) is str
    except:
        print("Found error in article", i, "- Passing")
        indices_to_drop.append(i)
        continue
    
    for keyword in keywords:
        if keyword in first300 or keyword in title:
            n_keywords += 1
            break
        else:
            pass
    
    if n_keywords == 0:
        indices_to_drop.append(i)
        
    if len(indices_to_drop) % 100 == 0:
        print("Marked", len(indices_to_drop), "articles for removal")
    
    if i % 10 == 0:
        print("Article", i)
        
print("Total:", len(indices_to_drop), "articles marked for removal.")

Marked 0 articles for removal
Article 0
Article 10
Article 20
Article 30
Article 40
Article 50
Article 60
Article 70
Article 80
Article 90
Article 100
Article 110
Article 120
Article 130
Article 140
Article 150
Article 160
Article 170
Article 180
Article 190
Article 200
Article 210
Article 220
Article 230
Article 240
Article 250
Article 260
Article 270
Article 280
Article 290
Article 300
Article 310
Article 320
Article 330
Article 340
Article 350
Article 360
Article 370
Article 380
Marked 100 articles for removal
Article 390
Article 400
Article 410
Article 420
Article 430
Article 440
Article 450
Article 460
Article 470
Article 480
Article 490
Article 500
Article 510
Article 520
Article 530
Article 540
Article 550
Article 560
Article 570
Article 580
Article 590
Article 600
Article 610
Article 620
Article 630
Article 640
Article 650
Article 660
Article 670
Article 680
Article 690
Article 700
Article 710
Article 720
Article 730
Article 740
Article 750
Article 760
Article 770
Article 780
A

Article 5980
Article 5990
Article 6000
Article 6010
Article 6020
Article 6030
Article 6040
Article 6050
Article 6060
Article 6070
Article 6080
Article 6090
Article 6100
Article 6110
Article 6120
Article 6130
Article 6140
Article 6150
Article 6160
Article 6170
Article 6180
Article 6190
Article 6200
Article 6210
Article 6220
Marked 1500 articles for removal
Article 6230
Article 6240
Article 6250
Article 6260
Article 6270
Article 6280
Article 6290
Article 6300
Article 6310
Article 6320
Article 6330
Article 6340
Article 6350
Article 6360
Article 6370
Article 6380
Article 6390
Article 6400
Article 6410
Article 6420
Article 6430
Article 6440
Article 6450
Article 6460
Article 6470
Article 6480
Article 6490
Article 6500
Article 6510
Article 6520
Article 6530
Article 6540
Article 6550
Marked 1600 articles for removal
Article 6560
Article 6570
Article 6580
Article 6590
Article 6600
Article 6610
Article 6620
Article 6630
Article 6640
Article 6650
Article 6660
Article 6670
Article 6680
Article 669

Article 15380
Article 15390
Article 15400
Article 15410
Article 15420
Article 15430
Article 15440
Article 15450
Article 15460
Article 15470
Article 15480
Article 15490
Article 15500
Article 15510
Article 15520
Article 15530
Article 15540
Article 15550
Article 15560
Article 15570
Article 15580
Article 15590
Article 15600
Article 15610
Article 15620
Article 15630
Article 15640
Article 15650
Article 15660
Article 15670
Article 15680
Article 15690
Article 15700
Article 15710
Article 15720
Article 15730
Article 15740
Article 15750
Marked 4000 articles for removal
Article 15760
Marked 4000 articles for removal
Article 15770
Article 15780
Article 15790
Article 15800
Article 15810
Article 15820
Article 15830
Article 15840
Article 15850
Article 15860
Article 15870
Article 15880
Article 15890
Article 15900
Article 15910
Article 15920
Article 15930
Article 15940
Article 15950
Article 15960
Article 15970
Article 15980
Article 15990
Article 16000
Article 16010
Article 16020
Article 16030
Article 16

Article 20560
Article 20570
Article 20580
Article 20590
Article 20600
Article 20610
Article 20620
Article 20630
Article 20640
Article 20650
Article 20660
Article 20670
Article 20680
Article 20690
Marked 5200 articles for removal
Article 20700
Article 20710
Article 20720
Article 20730
Article 20740
Article 20750
Article 20760
Article 20770
Article 20780
Article 20790
Article 20800
Article 20810
Article 20820
Article 20830
Article 20840
Article 20850
Article 20860
Article 20870
Article 20880
Article 20890
Article 20900
Article 20910
Article 20920
Article 20930
Article 20940
Article 20950
Article 20960
Article 20970
Article 20980
Article 20990
Article 21000
Article 21010
Article 21020
Article 21030
Article 21040
Article 21050
Article 21060
Article 21070
Article 21080
Article 21090
Marked 5300 articles for removal
Marked 5300 articles for removal
Marked 5300 articles for removal
Article 21100
Marked 5300 articles for removal
Marked 5300 articles for removal
Marked 5300 articles for removal

Article 24770
Article 24780
Article 24790
Marked 6400 articles for removal
Marked 6400 articles for removal
Marked 6400 articles for removal
Article 24800
Article 24810
Article 24820
Article 24830
Article 24840
Article 24850
Article 24860
Article 24870
Article 24880
Article 24890
Article 24900
Article 24910
Article 24920
Article 24930
Article 24940
Article 24950
Article 24960
Article 24970
Article 24980
Article 24990
Article 25000
Article 25010
Marked 6500 articles for removal
Article 25020
Article 25030
Article 25040
Article 25050
Article 25060
Article 25070
Article 25080
Article 25090
Article 25100
Article 25110
Article 25120
Article 25130
Article 25140
Marked 6600 articles for removal
Article 25150
Marked 6600 articles for removal
Marked 6600 articles for removal
Article 25160
Article 25170
Article 25180
Article 25190
Article 25200
Article 25210
Article 25220
Article 25230
Article 25240
Article 25250
Article 25260
Article 25270
Article 25280
Article 25290
Article 25300
Article 25310

In [54]:
articles_final = articles_final.drop(indices_to_drop)

In [56]:
articles_final = articles_final.reset_index()
articles_final = articles_final.drop('index', axis = 1)
articles_final.head()

,author,contents,description,publisher,source_url,timeStamp,title
0,Stripe.com,A complete payments platform engineered for gr...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support
1,Editorial Team,As it scrambles to serve a massively expanding...,As it scrambles to serve a massively expanding...,Finextra.com,https://www.finextra.com/newsarticle/31558/coi...,2018-01-23 00:01:00+00:00,Coinbase hires former Twitter exec to lead cus...
2,Scott Scanlon,So many cryptocurrencies. So much money to be ...,So many cryptocurrencies. So much money to be ...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/shou...,2018-01-23 00:03:12+00:00,Should you buy bitcoin? Or Ethereum? Or Dash? ...
3,http://www.dailymail.co.uk/home/search.html?s=...,By\n\nPress Association\n\t\nPublished:\n 19:...,The Tokyo-based firm has been awarded a paymen...,Daily Mail,http://www.dailymail.co.uk/wires/pa/article-53...,2018-01-23 00:05:47+00:00,World´s biggest Bitcoin exchange wins backing ...
4,Phil Glazer,Initial coin offerings (ICOs) are a regulatory...,Initial coin offerings (ICOs) are a regulatory...,Hackernoon.com,https://hackernoon.com/is-regulation-needed-fo...,2018-01-23 00:06:02+00:00,Is Regulation Needed for Institutional Investo...


In [57]:
articles_final.to_csv("final_articles.csv")